3D энкодер- декодер. Отличие от предыдущей версии - вместо растра точек на вход сети подать вектора лендмарков в виде набора координат и длин.

In [1]:
# from keras.datasets import mnist
import numpy as np
# from jaw_gen import *
import tensorflow as tf 
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Lambda
import cv2

from tqdm import tqdm
# основное отличие - Landmark_gen больше не генерит картинки. только вектора. 
# и у него теперь есть 3D рисовалка, которая есть независимый метод
from jaw_gen3d import Landmark_gen 

In [2]:
# готовим входные данные
inst_ =                 Landmark_gen()
data_len =              50000
dataset =               []
dataset_spoiled =       []
dataset_vec =           []
dataset_vec_spoiled =   []

for i in range(data_len):
    scale =  9 + np.random.sample()*3     # это для размера 200, диапазон от 14 до 19 для размера 400
    factor = 2.2 + np.random.sample()/4   # это для размера 200, диапазон 2.5 - 2.7 для размера 400 
    # name = f"scale {scale:.4}  factor {factor:.3}"
    vec, vec_spoiled = inst_.image_gen(scale=scale, factor=factor, spoiled=True, shiftX=True, shiftY=True)
    # dataset.append(img_)
    # dataset_spoiled.append(img_spoiled_)

    # собираем датасет из векторов 
    dataset_vec.append(vec)
    dataset_vec_spoiled.append(vec_spoiled)
# print (f"dataset_vec {dataset_vec[:1]}")
print (f"dataset_vec len {len(dataset_vec)} ")
print (f"dataset_vec_spoiled len {len(dataset_vec_spoiled)} ")

# преобразуем датасет в numpy массив
# dataset =               np.array(dataset,               dtype="float32")    # x_train = x_train.astype('float32') / 255. 
# dataset_spoiled =       np.array(dataset_spoiled,       dtype="float32") 
dataset_vec =           np.array(dataset_vec,           dtype="float32") 
dataset_vec_spoiled =   np.array(dataset_vec_spoiled,   dtype="float32") 

# dataset =           np.reshape(dataset,         (len(dataset), 200, 200, 1))
# dataset_spoiled =   np.reshape(dataset_spoiled, (len(dataset_spoiled), 200, 200, 1))

# разобьем датасет на тренировочную и тестовую части
set_divider_ = round(data_len*0.8)
# print (f"set_divider_ {set_divider_}")

# x_train = dataset[:set_divider_]
# x_test = dataset[set_divider_:]
# print (f"length {len(x_train)} - {len(x_test)}")

# x_train_spoiled = dataset_spoiled[:set_divider_]
# x_test_spoiled = dataset_spoiled[set_divider_:]

# векторный датасет
x_train_vec =   dataset_vec[:set_divider_]
x_test_vec =    dataset_vec[set_divider_:]
# print (f"length vec {len(x_train_vec)} - {len(x_test_vec)}")

x_train_vec_spoiled =   dataset_vec_spoiled[:set_divider_]
x_test_vec_spoiled =    dataset_vec_spoiled[set_divider_:]
# print (f"vec [0]{x_test_vec[0]}")
print (f"length vec         {len(x_train_vec)} - {len(x_test_vec)}")
print (f"length vec spoil   {len(x_train_vec_spoiled)} - {len(x_test_vec_spoiled)}")



dataset_vec len 50000 
dataset_vec_spoiled len 50000 
length vec         40000 - 10000
length vec spoil   40000 - 10000


In [3]:
def create_vec_ae3D():
    # 3d кодер - декодер содержит на входе 6*16=96 значений.
    dim_code = 16 # размерность кодированного слоя
    activation = 'elu' 
    # activation = 'linear'
    inputs = Input(96,)
    WITH_BN = True # go with batcNormalization

    # encoder
    x = Dense(96, activation=activation)(inputs)
    x= BatchNormalization()(x)      if WITH_BN else x
    x = Dense(96, activation=activation)(x)
    x= BatchNormalization()(x)      if WITH_BN else x
    code = Dense(dim_code, activation='linear')(x)

    # decoder
    input_encoded = Input((dim_code,))
    x = Dense(dim_code, activation=activation)(input_encoded)
    x= BatchNormalization()(x)      if WITH_BN else x
    x = Dense(96, activation=activation)(x)
    x= BatchNormalization()(x)      if WITH_BN else x
    out = Dense(96, activation='linear')(x)
    
    # Модели
    encoder = Model(inputs, code, name="encoder")
    decoder = Model(input_encoded, out, name="decoder")
    autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")
    return encoder, decoder, autoencoder

encoder, decoder, autoencoder = create_vec_ae3D()
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
# autoencoder.compile(Adam(0.003), 'mse')

# c_autoencoder.summary()

In [4]:
batch_size = 256

def create_denoising_model(autoencoder):
    
    inputs  = Input(batch_shape=(batch_size, 96))
    inputs_spoiled  = Input(batch_shape=(batch_size, 96)) # вход для кореженных зубьев

    denoiser_model = Model([inputs, inputs_spoiled], autoencoder(inputs_spoiled), name="denoiser")
    # return noiser, denoiser_model
    return denoiser_model


# noiser, denoiser_model = create_denoising_model(c_autoencoder)
denoiser_model = create_denoising_model(autoencoder)
denoiser_model.compile(optimizer='adam', loss='mean_squared_error')

In [5]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

def plot_digits(*args):
    # for a in args:
        # print (f"type args {type(a)}")
    args = [x.squeeze() for x in args]
    n = min([x.shape[0] for x in args])
    
    plt.figure(figsize=(2*n, 2*len(args)))
    for j in range(n):
            for i in range(len(args)):
                ax = plt.subplot(len(args), n, i*n + j + 1)
                plt.imshow(args[i][j])
                plt.gray()
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

    plt.show()

In [6]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

def plot_3d(*args):
    # 
    n = min([x.shape[0] for x in args]) # сколько столбцов
    plt.figure(figsize=(2*n, 2*len(args)))
    for j in range(n):
        for i in range(len(args)):
            pass #ax = plt.


In [7]:
# вместо фита будем делать  model.train_on_batch читобы смотреть как учится и куда движемся
def train_on_batch(vec, vec_spoil, batch_size=batch_size, epochs=100):
    # подровняем массив картинок и обучающих векторов так, чтоб np.reshape не ругался - на несовпадение размерности
    # сделаем его длину кратной размеру батча
    # print (f"x_train_vec.shape {x_train_vec.shape}")    # (8000, 16, 6)
    # print (f"vec shape {vec.shape}")                    # (8000, 16, 6)
    # print (f"vec_spoil shape {vec_spoil.shape}")        # (8000, 16, 6)

    # x =     x   [:len(x_train)//    batch_size*batch_size]
    vec =   vec [:len(x_train_vec)//batch_size*batch_size]
    # x =     np.reshape(x,   (-1, batch_size, 200, 200, 1))  # x_test  = np.reshape(x_test,  (len(x_test),  200, 200, 1))
    vec =   np.reshape(vec, (-1, batch_size, 96))           # x_test  = np.reshape(x_test,  (len(x_test),  200, 200, 1))
    
    # и массив кореженных тоже
    # x_spoil =   x_spoil     [:len(x_train_spoiled)      //batch_size*batch_size]
    vec_spoil = vec_spoil   [:len(x_train_vec_spoiled)  //batch_size*batch_size]
    # x_spoil =   np.reshape(x_spoil,     (-1, batch_size, 200, 200, 1)) 
    vec_spoil = np.reshape(vec_spoil,   (-1, batch_size, 96)) 

    # print (f"\nvec shape {vec.shape}")                  # (31, 256, 96)
    # print (f"vec_spoil shape {vec_spoil.shape}")        # (31, 256, 96)
    losses = [10e100] # потери
    for epoch in range(1, epochs+1):
        # print(f"Epoch {epoch}")
        for i, _ in enumerate(vec):
            # batch_x =           x[i]
            # batch_x_spoil =     x_spoil[i]    
            batch_vec =         vec[i] 
            batch_vec_spoiled = vec_spoil[i]

            loss = denoiser_model.train_on_batch([batch_vec, batch_vec_spoiled], batch_vec)

        # эпоха отучилась, построим что вышло.
        n = 10
        vecs = batch_vec                    # это последний батч в каждой эпохе.
        # print (f"1! vecs shape{vecs.shape}") # shape - (256, 64)
        noised_vecs = batch_vec_spoiled     
        # а картинки не участвуют в учении, количество бачей у них при делении датасета на бачи
        # с векторами разное, поэтому найти соответствующую картинку проблематично. 
        # noised_imgs = noiser.predict(imgs, batch_size=batch_size)

        # задача в том, чтобы картинке сопоставить спойленую картинку ( это не сложно из сгенерированной 
        # выборки сделать), но потом предиктить для этой картинки вектора. и рисовать их на новой картинке 
        # средствами opencv 
        encoded_vecs = encoder.predict(noised_vecs[:n],  batch_size=n)
        decoded_vecs = decoder.predict(encoded_vecs[:n], batch_size=n)

        # формируем картинки из предсказанных векторов
        vecs =              np.reshape(vecs,            (-1, 16, 6))
        noised_vecs =       np.reshape(noised_vecs,     (-1, 16, 6))
        predicted_vecs =    np.reshape(decoded_vecs,    (-1, 16, 6))
        
        # print (f"vecs shape{vecs.shape} vecs[0] {vecs[0]}")
        
        # imgs_fr_vec =           np.array([im for im in map(inst_.draw_3d, (noised_vecs,predicted_vecs) )])
        # imgs_spoil_fr_vec =     np.array([im for im in map(inst_.draw_3d, noised_vecs)])
        # imgs_fr_decoded_vec =   np.array([im for im in map(inst_.draw_3d, predicted_vecs)])

        # print (f"batch_x shape{     np.array(batch_x).shape}")      # (256, 200, 200, 1)
        # print (f"imgs_fr_vec shape{ np.array(imgs_fr_vec).shape}")  # (256, 200, 200)
        if (epoch%200 ==0) or (epoch in [1,2, 10, 30, 80]):
            print(f"Epoch {epoch} loss -{loss}")
        # рисуем картинки и сохраняем модели
        # if (epoch % 20 ==0) or (epoch in [1,2,3,5,10]):
        # if (epoch % 200 ==0) or (epoch in [10, 30, 80]):
        if (epoch % 200 ==0) and (epoch > 1000):
            # print(f'Epoch {epoch}')
            # plot_digits(batch_x[:n], batch_x_spoil[:n], imgs_fr_vec[:n], imgs_spoil_fr_vec[:n], imgs_fr_decoded_vec[:n])            
            # plot_digits(imgs_fr_vec[:n][0], imgs_spoil_fr_vec[:n], imgs_fr_decoded_vec[:n])
            ##########plot_digits(imgs_fr_vec[:n])
            # plot_digits(batch_x[:n], batch_x_spoil[:n])
            if loss < min(losses):
                losses.append(loss)
                # save only the best models
                denoiser_model.save(f'models3D/denoiser{epoch}ep_loss__{loss:.3}__.h5')
                decoder.save(f'models3D/decoder{epoch}ep_loss__{loss:.3}__.h5')
                encoder.save(f'models3D/encoder{epoch}ep_loss__{loss:.3}__.h5')
        
train_on_batch(x_train_vec, x_train_vec_spoiled, batch_size=batch_size, epochs=20000)


Epoch 1 loss -4191.8955078125
Epoch 2 loss -2926.989013671875
Epoch 10 loss -2.6505441665649414
Epoch 30 loss -0.0907314270734787
Epoch 80 loss -0.07411859184503555
Epoch 200 loss -0.062350623309612274
Epoch 400 loss -0.05787942558526993
Epoch 600 loss -0.046960871666669846
Epoch 800 loss -0.045594073832035065
Epoch 1000 loss -0.04529182240366936
Epoch 1200 loss -0.046661846339702606
Epoch 1400 loss -0.03993479907512665
Epoch 1600 loss -0.0402851477265358
Epoch 1800 loss -0.0402161180973053
Epoch 2000 loss -0.04240381717681885
Epoch 2200 loss -0.04063557833433151
Epoch 2400 loss -0.04101504385471344
Epoch 2600 loss -0.042272910475730896
Epoch 2800 loss -0.039403460919857025
Epoch 3000 loss -0.039863407611846924
Epoch 3200 loss -0.03948039188981056
Epoch 3400 loss -0.038855426013469696
Epoch 3600 loss -0.03835449367761612
Epoch 3800 loss -0.039333101361989975
Epoch 4000 loss -0.037515632808208466
Epoch 4200 loss -0.040841057896614075
Epoch 4400 loss -0.03983033820986748
Epoch 4600 loss 

In [8]:
# c_autoencoder.save(f'models/c_autoencoder.h5'


encoded_imgs[:20]